In [1]:
import keras
print(keras.__version__)
import tensorflow
print(tensorflow.__version__)
import numpy as np
print(np.__version__)

from keras_tqdm import TQDMNotebookCallback
# ...fit(verbose = 0, callbacks=[TQDMNotebookCallback()])


from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K

from keras.optimizers import Adam

/Users/alecx/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


2.2.4
1.12.0
1.15.4


In [2]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [3]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.BatchNormalization())
model.add(layers.Dense(9, activation='softmax'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1280)              5120      
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 11529     
Total params: 2,274,633
Trainable params: 2,237,961
Non-trainable params: 36,672
_________________________________________________________________


In [5]:
# define metric
from balancedAccuracy import balancedAccuracy
num_classes = 9
bacc_metric = balancedAccuracy(num_classes)

In [6]:
# plot_confusion_matrix function
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

class_names = ["MEL", "NV", "BCC", "AK", "KL", "DF", "VASC", "SCC", "UNK"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(np.mean(np.diag(cm)))
        
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    # get balanced accuracy
    return np.mean(np.diag(cm))
    


In [7]:
data = np.load("/Users/alecx/Downloads/AWS-LESIONDATA-2019.npz")

imageList = data["imageList"]
targetList = data["targetList"]
imageValList = data["imageValList"]
targetValList = data["targetValList"]
testList = data["testList"]
targetTestList = data["targetTestList"]

In [9]:
# Get only non-augmented images

imageList = imageList[::5]
targetList = targetList[::5]

In [10]:
def valMatrix():
    y_test = targetValList.copy()
    y_pred = model.predict(imageValList)
    y_pred = y_pred.argmax(1)
    y_test = y_test.argmax(1)

    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    plt.figure()
    print("Balanced Accuracy: "+ str(plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized Validation confusion matrix')))
    plt.show()

def trainMatrix():
    y_test = targetList.copy()
    y_pred = model.predict(imageList)
    y_pred = y_pred.argmax(1)
    y_test = y_test.argmax(1)

    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    plt.figure()
    print("Balanced Accuracy: "+ str(plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized Training confusion matrix')))
    plt.show()
    
def testMatrix():
    y_test = targetTestList.copy()
    y_pred = model.predict(testList)
    y_pred = y_pred.argmax(1)
    y_test = y_test.argmax(1)

    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    plt.figure()
    print("Balanced Accuracy: "+ str(plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized Training confusion matrix')))
    plt.show()

In [11]:
# get sample weights for training (class_weight doesn't work with one-hot encoding)
weight_dict = {0:5.6050885 ,   1:1.96776699,   2:7.61954887,  3:29.28901734,
         4:9.65142857, 5:105.5625,  6:99.35294118,  7:40.21428571,
                8:0}
temp = targetList.argmax(1)
sample_weights = np.array(list(map(weight_dict.get, temp)))

# get sample weights for validation


temp = targetValList.argmax(1)
sample_weights_val = np.array(list(map(weight_dict.get, temp)))

In [12]:
import json

# Set up Online Data Augmentation

In [13]:
from keras.preprocessing.image import ImageDataGenerator

# create data generator
datagen = ImageDataGenerator(width_shift_range=[-50,50], height_shift_range=0.2,
                            horizontal_flip=True, vertical_flip=True,
                            rotation_range=360,
                            brightness_range=[0.8,1.2], zoom_range=[0.9,1.1],
                            preprocessing_function=preprocess_input)

it = datagen.flow(imageList, targetList, batch_size=32)

# Find Optimal Learning Rate

In [22]:
import matplotlib.pyplot as plt
import keras.backend as K
from keras.callbacks import Callback


class LRFinder(Callback):
    
    '''
    A simple callback for finding the optimal learning rate range for your model + dataset. 
    
    # Usage
        ```python
            lr_finder = LRFinder(min_lr=1e-5, 
                                 max_lr=1e-2, 
                                 steps_per_epoch=np.ceil(epoch_size/batch_size), 
                                 epochs=3)
            model.fit(X_train, Y_train, callbacks=[lr_finder])
            
            lr_finder.plot_loss()
        ```
    
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        epochs: Number of epochs to run experiment. Usually between 2 and 4 epochs is sufficient. 
        
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: https://arxiv.org/abs/1506.01186
    '''
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=None, epochs=None):
        super().__init__()
        
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_iterations = steps_per_epoch * epochs
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
 
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        plt.show()
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')
        plt.show()

In [23]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy',
              metrics=[bacc_metric.balanced_acc])

lr_finder = LRFinder(min_lr=1e-5, 
                     max_lr=1e-2, 
                     steps_per_epoch=np.ceil(len(imageList)/32), 
                     epochs=3)
model.fit_generator(it, steps_per_epoch=np.ceil(len(imageList)/32),
                    epochs=5, verbose=0,
                    class_weight=weight_dict,
                    validation_data=(imageValList, targetValList, sample_weights_val),
                    callbacks=[lr_finder])

lr_finder.plot_loss()

KeyboardInterrupt: 

# Train Model

In [14]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',
              metrics=[bacc_metric.balanced_acc])

In [18]:
model.fit_generator(it, steps_per_epoch=np.ceil(len(imageList)/32), 
                    epochs=5, verbose=0,
                    class_weight=weight_dict,
                    validation_data=(imageValList, targetValList, sample_weights_val),
                    callbacks = [TQDMNotebookCallback(leave_inner=True)])

KeyboardInterrupt: 

In [ ]:
model.fit(imageList, targetList, batch_size=32, epochs=3, verbose=0,
          sample_weight=sample_weights,
          validation_data=(imageValList, targetValList, sample_weights_val), initial_epoch=0,
          callbacks = [TQDMNotebookCallback(leave_inner=True)])


In [ ]:
model.fit(imageList, targetList, batch_size=32, epochs=3, verbose=0,
          sample_weight=sample_weights,
          validation_data=(imageValList, targetValList, sample_weights_val), initial_epoch=0,
          callbacks = [TQDMNotebookCallback(leave_inner=True)])

In [ ]:
model.save("dnet201-inet-3ep")

In [ ]:
valMatrix()

In [ ]:
testMatrix()

In [ ]:
# TEMP
from keras.models import load_model
model = load_model('/vol30/resnet-final/checkpoint1',
                   custom_objects={'balanced_acc': bacc_metric.balanced_acc})